In [ ]:
%pip install openai
%pip install docarray
%pip install tiktoken

In [2]:
import os
from getpass import getpass

os.environ['OPENAI_API_KEY'] = getpass('Enter your OpenAI API Key: ')

In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

messages = template.format_messages(
    name="Bob",
    user_input="What is your name?"
)

print(messages)

# model = ChatOpenAI()
# model(messages)

chain = messages | model | output_parser

# chain.invoke()

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import HumanMessagePromptTemplate
from langchain.schema.messages import SystemMessage

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that re-writes the user's text to "
                "sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

llm = ChatOpenAI(model_name="gpt-3.5-turbo")

output_parser = StrOutputParser()

chain = chat_template | llm | output_parser

chain.invoke({'text': 'this is hard'})

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

template="You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

llm = ChatOpenAI()

chain = chat_template | llm

chain.invoke({'input_language': 'English', 'output_language': 'French', 'text': 'Hello, how are you?'})

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

template="You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

model = ChatOpenAI()

chain = chat_prompt | model

chain.invoke({'input_language': 'English', 'output_language': 'French', 'text': 'Hello, how are you?'})

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-3.5-turbo")
chain = (
    {"topic": RunnablePassthrough()} 
    | prompt
    | model
    | output_parser
)

chain.invoke("ice cream")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at terminal", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("what do bears eat?")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain.schema.output_parser import StrOutputParser

# Generate system and human messages
messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(content="What is the purpose of model regularization?"),
]

prompt = ChatPromptTemplate.from_messages(messages)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

# Stream the chain
for chunk in chain.stream("ice cream"):
    print(chunk, end="", flush=True)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

template="You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

model = ChatOpenAI()

chain = chat_prompt | model

chain.invoke({'input_language': 'English', 'output_language': 'French', 'text': 'Hello, how are you?'})

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Define the system and human message templates
system_message = ChatPromptTemplate.from_template("this is a system message")
human_message = ChatPromptTemplate.from_template("this is a human message: {input}")

# Create the OpenAI model and output parser
model = ChatOpenAI()
output_parser = StrOutputParser()

# Create the chain
chain = (
    {"input": RunnablePassthrough()} 
    | system_message
    | human_message
    | model
    | output_parser
)

# Invoke the chain with a string input
chain.invoke("your input string here")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

# Define system and human message templates
system_message = SystemMessage(content="Write a cute 3-line poem about a topic")
human_message_template = "Here's my topic: {input}"
human_message_prompt = HumanMessage.template(human_message_template)

# Combine messages into a chat prompt
prompt = ChatPromptTemplate.from_messages([system_message, human_message_prompt])

# Initialize the model and output parser
model = ChatOpenAI()
output_parser = StrOutputParser()

# Create the chain
chain = (
    {"input": RunnablePassthrough()} 
    | prompt
    | model
    | output_parser
)

# Invoke the chain with an input string
chain.invoke("bunnies")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Define system and human message templates
system_message_prompt = SystemMessagePromptTemplate.from_template("Write a cute 3-line poem on the requested topic")
human_message_prompt = HumanMessagePromptTemplate.from_template("{input}")

# Create a chat prompt from the message templates
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# Initialize the OpenAI model
model = ChatOpenAI()

# Define the output parser
output_parser = StrOutputParser()

# Create the chain
chain = (
    {"input": RunnablePassthrough()} 
    | chat_prompt
    | model
    | output_parser
)

# Invoke the chain with an input string
chain.invoke("Bunnies")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

prompt = ChatPromptTemplate.from_template("tell a joke about {foo}")

model = ChatOpenAI(model="gpt-3.5-turbo")

functions = [
    {
        "name": "joke",
        "description": "A joke",
        "parameters": {
            "type": "object",
            "properties": {
                "setup": {"type": "string", "description": "The setup for the joke"},
                "punchline": {
                    "type": "string",
                    "description": "The punchline for the joke",
                },
            },
            "required": ["setup", "punchline"],
        },
    }
]

chain = (
    {"foo": RunnablePassthrough()}
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

chain.invoke("bears")

In [ ]:
from operator import itemgetter

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what country is the city {city} in? respond in {language}"
)

model = ChatOpenAI()

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"person": "obama", "language": "spanish"})

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate

prompt1 = ChatPromptTemplate.from_template(
    "generate a {attribute} color. Return the name of the color and nothing else:"
)
prompt2 = ChatPromptTemplate.from_template(
    "what is a fruit of color roughly: {color}. Return the name of the fruit and nothing else:"
)
prompt3 = ChatPromptTemplate.from_template(
    "what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:"
)
prompt4 = ChatPromptTemplate.from_template(
    "What is the color of {fruit} and the flag of {country}?"
)

model = ChatOpenAI(model="gpt-3.5-turbo")
model_parser = model | StrOutputParser()

color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
)
color_to_fruit = prompt2 | model_parser
color_to_country = prompt3 | model_parser
question_generator = (
    color_generator | {"fruit": color_to_fruit, "country": color_to_country} | prompt4
)

question_generator.invoke("cold")

In [5]:
from operator import itemgetter

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser


planner = (
    ChatPromptTemplate.from_template("Generate a brief argument about: {input}")
    | ChatOpenAI()
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

arguments_for = (
    ChatPromptTemplate.from_template(
        "List 3 pros or positive aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)
arguments_against = (
    ChatPromptTemplate.from_template(
        "List 3 cons or negative aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "Review your original response (below), and update it based upon the pros and cons.{original_response}"),
            ("human", "Pros:\n{results_1}\n\nCons:\n{results_2}"),
            ("system", "Generate a final response given the critique"),
        ]
    )
    | ChatOpenAI()
    | StrOutputParser()
)

chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

chain.invoke({"input": "scrum"})

"Scrum is a highly effective project management framework that offers numerous benefits, but it's important to consider the potential drawbacks as well. \n\nOne of the main advantages of Scrum is its emphasis on collaboration and teamwork. By promoting open communication and regular interaction among project stakeholders, Scrum fosters a sense of shared responsibility and leads to improved problem-solving and productivity. \n\nAnother benefit is the adaptability and flexibility that Scrum provides. Its iterative approach allows teams to continuously inspect and adapt their processes and product, enabling them to respond quickly to changes or new requirements. This flexibility facilitates innovation and experimentation, leading to better outcomes. \n\nScrum also encourages continuous improvement, with its focus on reflection and refinement at the end of each sprint. This culture of learning and innovation ensures that teams are constantly striving to enhance their processes and deliver 

In [ ]:
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain.pydantic_v1 import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")

parser = PydanticOutputParser(pydantic_object=Actor)

prompt_template = """
Answer the user query.\n
{format_instructions}\n
{query}\n
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

model = ChatOpenAI()

chain = (
    {"query": RunnablePassthrough()}
    | prompt
    | model
    | parser
)

chain.invoke("Generate the filmography for a random actor who was in Breaking Bad.")

In [ ]:
from typing import List
from langchain.output_parsers import PydanticOutputParser
from langchain.pydantic_v1 import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

class Event(BaseModel):
    event_name: str = Field(description="name of the event")
    event_description: str = Field(description="description of the event")

parser = PydanticOutputParser(pydantic_object=Event)

prompt_template = """
Return an event that happened on this date: {date_string}\n
{format_instructions}\n
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["date_string"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

model = ChatOpenAI()

chain = (
    {"date_string": RunnablePassthrough()}
    | prompt
    | model
    | parser
)

chain.invoke("1989-11-09")